In [ ]:
import os
import json
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

# Thiết lập đường dẫn
PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_DIR = os.path.join(PROJECT_DIR, "data")
RAW_DIR = os.path.join(DATA_DIR, "raw")
MODEL_DIR = os.path.join(PROJECT_DIR, "models")
RESULT_DIR = os.path.join(PROJECT_DIR, "result")

# Tạo thư mục result nếu chưa tồn tại
os.makedirs(RESULT_DIR, exist_ok=True)

# Tải model đã train
model = YOLO(os.path.join(MODEL_DIR, 'number_detection_best.pt'))

# Hàm để vẽ bounding boxes và lưu kết quả
def process_image(image_path, output_path, json_path):
    # Thực hiện dự đoán
    results = model(image_path)[0]
    
    # Lấy ảnh gốc
    img = Image.open(image_path)
    
    # Vẽ bounding boxes
    fig, ax = plt.subplots(1, 1, figsize=(12, 9))
    ax.imshow(img)
    
    detections = []
    
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0]
        class_id = int(box.cls)
        conf = float(box.conf)
        label = f"{results.names[class_id]} {conf:.2f}"
        
        rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x1, y1, label, color='red', fontsize=12, verticalalignment='top')
        
        detections.append({
            "class": results.names[class_id],
            "confidence": conf,
            "bbox": [float(x1), float(y1), float(x2), float(y2)]
        })
    
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()
    
    # Lưu kết quả dưới dạng JSON
    with open(json_path, 'w') as f:
        json.dump(detections, f, indent=2)

# Xử lý tất cả ảnh trong thư mục test
test_dir = os.path.join(RAW_DIR, 'test')
for img_file in os.listdir(test_dir):
    if img_file.endswith('.jpg'):
        image_path = os.path.join(test_dir, img_file)
        output_path = os.path.join(RESULT_DIR, f"{os.path.splitext(img_file)[0]}_result.jpg")
        json_path = os.path.join(RESULT_DIR, f"{os.path.splitext(img_file)[0]}_result.json")
        
        process_image(image_path, output_path, json_path)
        print(f"Đã xử lý: {img_file}")

print("Hoàn tất xử lý tất cả ảnh test.")